In [9]:
!pip install diffusers transformers accelerate torch gradio

# Import necessary libraries

In [10]:
import torch
import gradio as gr
from diffusers import (
    StableDiffusionPipeline,
    EulerDiscreteScheduler,
    DDIMScheduler,
    PNDMScheduler,
    LMSDiscreteScheduler,
    DPMSolverMultistepScheduler
)
from PIL import Image

# Load the Model

In [11]:
# === Load Model Function ===
def load_pipeline(scheduler_name):
    model_id = "runwayml/stable-diffusion-v1-5"

    # Scheduler selection
    if scheduler_name == "Euler":
        scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
    elif scheduler_name == "DDIM":
        scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler")
    elif scheduler_name == "PNDM":
        scheduler = PNDMScheduler.from_pretrained(model_id, subfolder="scheduler")
    elif scheduler_name == "LMS":
        scheduler = LMSDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
    elif scheduler_name == "DPM":
        scheduler = DPMSolverMultistepScheduler.from_pretrained(model_id, subfolder="scheduler")
    else:
        raise ValueError(f"Unknown scheduler: {scheduler_name}")

    # Load pipeline
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id,
        scheduler=scheduler,
        torch_dtype=torch.float32  # use float16 if using GPU
    )
    pipe = pipe.to("cpu")  # use "cuda" for GPU
    return pipe

# Generate Image

In [12]:
# === Generate Image Function ===
def generate_image(prompt, negative_prompt, height, width, steps, guidance, seed, scheduler_name):
    pipe = load_pipeline(scheduler_name)
    generator = torch.manual_seed(int(seed)) if seed else None

    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        height=int(height),
        width=int(width),
        num_inference_steps=int(steps),
        guidance_scale=float(guidance),
        generator=generator
    ).images[0]

    return image

# User Interface

In [ ]:


# === Gradio UI ===
with gr.Blocks() as interface:
    gr.Markdown("## 🖼️ Gen AI Text-to-Image Generator using Stable Diffusion with scheduler")

    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="Prompt", value="a futuristic city under the sea")
            negative_prompt = gr.Textbox(label="Negative Prompt", value="blurry, low quality")
            scheduler_dropdown = gr.Dropdown(
                choices=["Euler", "DDIM", "PNDM", "LMS", "DPM"],
                value="Euler",
                label="Scheduler"
            )
            height = gr.Slider(256, 768, value=512, step=64, label="Height")
            width = gr.Slider(256, 768, value=512, step=64, label="Width")
            steps = gr.Slider(10, 100, value=40, step=5, label="Inference Steps")
            guidance = gr.Slider(1.0, 15.0, value=8.5, step=0.5, label="Guidance Scale")
            seed = gr.Textbox(label="Seed", value="1234")
            generate_btn = gr.Button("Generate Image")

        with gr.Column():
            output_image = gr.Image(label="Generated Image")

    generate_btn.click(
        fn=generate_image,
        inputs=[prompt, negative_prompt, height, width, steps, guidance, seed, scheduler_dropdown],
        outputs=output_image
    )

interface.launch(share=True,debug = True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ef8619ab95626918ec.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
